<a href="https://colab.research.google.com/github/Malikk1997/Knee-Osteoarthritis/blob/main/Knee_Osteoarthritis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# Update this path to the path of your dataset in Google Drive
dataset_path = '/content/drive/My Drive/MedicalExpert-I'

# List the files in your dataset directory
print("Files in dataset directory:")
print(os.listdir(dataset_path))


Files in dataset directory:
['1Doubtful', '3Moderate', '2Mild', '4Severe', '0Normal']


In [ ]:
'''
this code is used for medical expert 2 after we implement the 1st one succuessfully
import os

# Update this path to the path of your dataset in Google Drive
dataset_path = '/content/drive/My Drive/MedicalExpert-II'

# List the files in your dataset directory
print("Files in dataset directory:")
print(os.listdir(dataset_path)) '''

'\nthis code is used for medical expert 2 after we implement the 1st one succuessfully\nimport os\n\n# Update this path to the path of your dataset in Google Drive\ndataset_path = \'/content/drive/My Drive/MedicalExpert-II\'\n\n# List the files in your dataset directory\nprint("Files in dataset directory:")\nprint(os.listdir(dataset_path)) '

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

In [ ]:
import tensorflow as tf

batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  dataset_path,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode='categorical'
)

validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
  dataset_path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode='categorical'
)


Found 1650 files belonging to 5 classes.
Using 1320 files for training.
Found 1650 files belonging to 5 classes.
Using 330 files for validation.


In [ ]:
import tensorflow as tf

# Assuming train_ds and validation_ds are already defined as shown in your example

# Determine the number of validation batches
# Note: The cardinality method returns the number of batches in the dataset
val_batches = validation_ds.cardinality().numpy()

# Calculate the split (in this example, 50% for validation, 50% for testing)
val_size = int(val_batches * 0.5)

# Split the validation dataset
val_ds = validation_ds.take(val_size)
test_ds = validation_ds.skip(val_size)

# Optionally, you can cache, shuffle, and prefetch the datasets for optimal performance
AUTOTUNE = tf.data.AUTOTUNE
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)



In [ ]:
from tensorflow.keras.applications import EfficientNetB7

base_model = EfficientNetB7(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze the base model to prevent weights from being updated during the first phase of training
base_model.trainable = False


258076736/258076736 [==============================] - 3s 0us/step


In [ ]:
from tensorflow.keras import layers, models, regularizers

model = models.Sequential([
  base_model,
  layers.GlobalAveragePooling2D(),
  layers.Dense(1024, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(5, activation='softmax')
  # Assuming 5 categories for severity
])


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
epochs = 50  # Adjust based on your dataset size and desired training time

history = model.fit(
  train_ds,
  validation_data=validation_ds,
  epochs=epochs
)


Epoch 1/50
42/42 [==============================] - 61s 729ms/step - loss: 1.2897 - accuracy: 0.4795 - val_loss: 0.9743 - val_accuracy: 0.6091
Epoch 2/50
42/42 [==============================] - 20s 472ms/step - loss: 0.9089 - accuracy: 0.6333 - val_loss: 0.8281 - val_accuracy: 0.6515
Epoch 3/50
42/42 [==============================] - 19s 454ms/step - loss: 0.8091 - accuracy: 0.6856 - val_loss: 0.8068 - val_accuracy: 0.6636
Epoch 4/50
42/42 [==============================] - 20s 469ms/step - loss: 0.7713 - accuracy: 0.6894 - val_loss: 0.7289 - val_accuracy: 0.7121
Epoch 5/50
42/42 [==============================] - 19s 456ms/step - loss: 0.6934 - accuracy: 0.7129 - val_loss: 0.8480 - val_accuracy: 0.6576
Epoch 6/50
42/42 [==============================] - 19s 465ms/step - loss: 0.6586 - accuracy: 0.7379 - val_loss: 0.7053 - val_accuracy: 0.7182
Epoch 7/50
42/42 [==============================] - 19s 456ms/step - loss: 0.6186 - accuracy: 0.7515 - val_loss: 0.6601 - val_accuracy: 0.7515

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_ds)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


6/6 [==============================] - 3s 304ms/step - loss: 0.5903 - accuracy: 0.8118
Test Loss: 0.5903204679489136
Test Accuracy: 0.8117647171020508
